<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Andrés Hernández Jiménez, Oscar Flores Hernández.

**Fecha:** 28 de abril del 2021.

**Expediente** : 717895, 715029 .
**Profesor:** Oscar David Jaramillo Zuluaga.


# Tarea 10 : Clase 23

# Tarea

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$. Realizar la simulación en base a la siguiente tabla:
![imagen.png](attachment:imagen.png)

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Trapezoidal_approach`. Concluya. **(30 puntos)**

In [1]:
#importamos las librerias
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
%matplotlib inline
from time import time
from datetime import date
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 21)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [8]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    """
    Expresión de la solución de la ecuación de Black-Scholes
    St = S0*exp((r-sigma^2/2)*t+ sigma*DeltaW)
    
    Parámetros
    ---------
    mu    : Tasa libre de riesgo
    sigma : Desviación estándar de los rendimientos
    S0    : Precio inicial del activo subyacente
    NbTraj: Cantidad de trayectorias a simular
    NbStep: Número de días a simular
    """
    # Datos para la fórmula de St
    nu = mu-(sigma**2)/2
    DeltaT = 1/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    
    # Se obtiene --> Ln St = Ln S0+ nu*DeltaT + sigma*DeltaW
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    
    # Se utiliza cumsum por que se quiere simular los precios iniciando desde S0
    LogSt = np.cumsum(concat,axis=1)
    # Se obtienen los precios simulados para los NbStep fijados
    St = np.exp(LogSt)
    # Vector con la cantidad de días simulados
    t = np.arange(0,NbStep)

    return St.T,t

def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]

In [84]:
# Función donde se almacenan todos los resultados
def Trapezoidal_approach(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                         NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                         sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',
                         tipo: 'call o put',
                        flag=None):
    # Resolvemos la ecuación de black scholes para obtener los precios
    Tt= time() #registro del tiempo
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    h=T/NbStep
    # Obtenemos los precios promedios (trapecio)
    Average_t = np.cumsum(np.multiply(prices,(2+r*h+np.random.randn(NbStep,NbTraj)*sigma)))*h/(2*T)
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]),index=t)
    if tipo=='call':
        # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
        call = pd.DataFrame({'Prima':np.exp(-r*T) \
                             *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
        confianza = 0.95
        sigma_est = call.sem().Prima
        mean_est = call.iloc[-1].Prima
        i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        return [call.iloc[-1].Prima, i1[0], i1[1], np.abs(i1[0] - i1[1]), time()-Tt]    
    
    else:
        put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
        confianza = 0.95
        sigma_est = put.sem().Prima
        mean_est = put.iloc[-1].Prima
        i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        return [put.iloc[-1].Prima, i1[0], i1[1], np.abs(i1[0] - i1[1]), time()-Tt]

In [85]:
def Riemann_approach(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad',T:'Tiempo de cierre del contrato en años',tipo:'call o put',
                    flag=None):
    Tt= time() #registro del tiempo
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Creamos el segundo DataFrame
    mult= pd.DataFrame(2+r*(T/NbStep)+np.random.randn(NbTraj,NbStep)*sigma).T
    # Multiplicamos los dataframes
    prices_mult= prices*mult
    # Hacemos la sumatoria
    sumprices_mult = np.cumsum(prices_mult)
    # Multiplicamos por h/2 
    sumatoria = sumprices_mult*(1/NbStep)*(1/2)*(1/T)
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    if tipo=='call':
        # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
        call = pd.DataFrame({'Prima':np.exp(-r*T) \
                             *np.fmax(sumatoria-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
        confianza = 0.95
        sigma_est = call.sem().Prima
        mean_est = call.iloc[-1].Prima
        i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        return [call.iloc[-1].Prima, i1[0], i1[1], np.abs(i1[0] - i1[1]), time()-Tt]

    else:
        put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-sumatoria,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
        # intervalos de confianza
        confianza = 0.95
        sigma_est = put.sem().Prima
        mean_est = put.iloc[-1].Prima
        i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        return [put.iloc[-1].Prima, i1[0], i1[1], np.abs(i1[0] - i1[1]), time()-Tt]

In [86]:
S0=100 #precio inicial
K=100 #precio de ejercicio
r=0.10 #tasa rf
sigma=0.20 #volatilidad
T=1 #plazo

In [115]:
NbTraj=[1000,5000,10000,50000,100000,500000,1000000]
NbStep=[10,50,100]

In [116]:
montecarlo = [[N_x, N_y] + Riemann_approach(K,r,S0,N_x,N_y,sigma,T,"call") for N_x in NbTraj for N_y in NbStep]
MonteCarlo = pd.DataFrame(data=montecarlo)
MonteCarlo.columns = ['Trayectorias', 'Numero de pasos', 'Aproximación', 'L. inferior', 'L. superior', 'distancia al 95%', 'Tiempo de ejecucion (s)']
MonteCarlo

,Trayectorias,Numero de pasos,Aproximación,L. inferior,L. superior,distancia al 95%,Tiempo de ejecucion (s)
0,1000,10,6.743,5.418,8.069,2.650,0.010
1,1000,50,6.782,6.385,7.180,0.795,0.013
2,1000,100,7.195,6.913,7.477,0.564,0.007
3,5000,10,6.849,5.505,8.193,2.687,0.010
4,5000,50,6.963,6.552,7.374,0.821,0.030
5,5000,100,6.987,6.713,7.262,0.549,0.060
6,10000,10,6.984,5.613,8.355,2.741,0.020
7,10000,50,7.072,6.652,7.491,0.839,0.070
8,10000,100,6.978,6.703,7.252,0.549,0.100
9,50000,10,6.890,5.539,8.242,2.703,0.110


In [117]:
trapecio = [[N_x, N_y] + Trapezoidal_approach(K,r,S0,N_x,N_y,sigma,T,'call') for N_x in NbTraj for N_y in NbStep]
Trapecium = pd.DataFrame(data=trapecio)
Trapecium.columns = ['Trayectorias', 'Numero de pasos', 'Aproximación', 'L. inferior', 'L. superior', 'distancia al 95%', 'Tiempo de ejecucion (s)']
Trapecium

,Trayectorias,Numero de pasos,Aproximación,L. inferior,L. superior,distancia al 95%,Tiempo de ejecucion (s)
0,1000,10,6.935,5.574,8.296,2.722,0.002
1,1000,50,6.714,6.321,7.106,0.785,0.015
2,1000,100,7.338,7.048,7.628,0.580,0.015
3,5000,10,7.123,5.724,8.521,2.797,0.015
4,5000,50,6.924,6.516,7.331,0.815,0.035
5,5000,100,6.818,6.552,7.085,0.533,0.054
6,10000,10,6.842,5.500,8.184,2.684,0.020
7,10000,50,6.974,6.563,7.385,0.822,0.061
8,10000,100,7.056,6.778,7.334,0.556,0.099
9,50000,10,6.868,5.521,8.215,2.694,0.110


Comparando los resultados de los dos DataFrame, podemos encontrar que cuando la cantidad de simulaciones es mayor se aproximan mejor al resultado esperado, del mismo modo, mientras más pasos se tomen se obtendrán mejores resultados. Cabe resaltar que parece que hay un 'límite' de aproximación en el número de pasos establecido. Es decir, si se mantiene la misma cantidad de pasos no será suficiente con aumentar la cantidad de simulaciones, sino que habrá que aumentar ambas. 

En el ejemplo siguiente se hace una propuesta de lo que sería aumentar la cantidad de pasos sin llegar a 1M de simulaciones. Notese que todos los siguientes obtienen mejores resultados en cuanto a varianza, sin embargo el tiempo de ejecución es menor en todos ellos (a excepción del último caso.) 

In [121]:
NbTraj=[100000,500000]
NbStep=[200, 500]

xtra = [[N_x, N_y] + Trapezoidal_approach(K,r,S0,N_x,N_y,sigma,T,'call') for N_x in NbTraj for N_y in NbStep]
Extra = pd.DataFrame(data=xtra)
Extra.columns = ['Trayectorias', 'Numero de pasos', 'Aproximación', 'L. inferior', 'L. superior', 'distancia al 95%', 'Tiempo de ejecucion (s)']
Extra

,Trayectorias,Numero de pasos,Aproximación,L. inferior,L. superior,distancia al 95%,Tiempo de ejecucion (s)
0,100000,200,7.049,6.859,7.239,0.380,1.856
1,100000,500,7.066,6.948,7.185,0.237,4.303
2,500000,200,7.045,6.855,7.235,0.380,9.343
3,500000,500,7.038,6.920,7.155,0.235,23.462
